In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install faiss-cpu --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 260.8 MB/s eta 0:00:00a 0:00:01


In [4]:
# ─── Cell 1: Imports ───────────────────────────────────────────────────────────
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import os


In [5]:
# ─── Cell 2: Load & normalize your full JSON ───────────────────────────────────
# ▶️ Change path to wherever your 9000-blog file lives under /kaggle/input
with open("/kaggle/input/fullblogs/scraped_blogs_final.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Pull real text into "text" (fallback body→"")
for item in data:
    item["text"] = item.pop("content", item.pop("body", ""))


In [6]:
# ─── Cell 3: Build a pandas DataFrame ─────────────────────────────────────────
df = pd.DataFrame([
    {"title": b.get("title",""), "url": b.get("url",""), "text": b["text"]}
    for b in data
])


In [7]:
# ─── Cell 4: Embed & build your FAISS index ──────────────────────────────────
# 1) Embed all documents
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df["text"].tolist(),
                          batch_size=32,
                          show_progress_bar=True)

# 2) Build a normalized inner-product (cosine) FAISS index
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)
faiss.normalize_L2(embeddings)
index.add(embeddings)

# 3) Persist to disk
os.makedirs("indexes", exist_ok=True)
faiss.write_index(index,       "indexes/faiss.index")
with open("indexes/metadata.pkl","wb") as f:
    pickle.dump(df, f)

print("Done. Indexed", len(df), "documents.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/279 [00:00<?, ?it/s]

Done. Indexed 8925 documents.
